Lab 3

Task:
1. Write a program for planning a path from the initial point to the final point bypassing obstacles using the potential field method.
2. Independently determine the initial point of movement, the final point, and obstacles in the workspace.
3. The program code must reflect all the steps for constructing the potential field of the workspace, the order of calculating the gradient, and determining the subsequent direction of movement.
4. Illustrate achieved results.

In [10]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import SE3
from swift import Swift

1. Loading the manipulator model Panda


In [11]:
robot = rtb.models.DH.Puma560()
env = Swift()
env.launch(realtime=True)
env.add(robot)

0

connection handler failed
Traceback (most recent call last):
  File "c:\Users\Vladr\AppData\Local\Programs\Python\Python38\lib\site-packages\websockets\legacy\protocol.py", line 953, in transfer_data
    message = await self.read_message()
  File "c:\Users\Vladr\AppData\Local\Programs\Python\Python38\lib\site-packages\websockets\legacy\protocol.py", line 1023, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
  File "c:\Users\Vladr\AppData\Local\Programs\Python\Python38\lib\site-packages\websockets\legacy\protocol.py", line 1098, in read_data_frame
    frame = await self.read_frame(max_size)
  File "c:\Users\Vladr\AppData\Local\Programs\Python\Python38\lib\site-packages\websockets\legacy\protocol.py", line 1155, in read_frame
    frame = await Frame.read(
  File "c:\Users\Vladr\AppData\Local\Programs\Python\Python38\lib\site-packages\websockets\legacy\framing.py", line 70, in read
    data = await reader(2)
  File "c:\Users\Vladr\AppData\Local\Programs\Pyth

2. Definition of working poses 
Initial pose (q = [θ1, θ2, d3, θ4, θ5, θ6])

In [18]:
start_pose = SE3(0.5, -0.3, 0.2)

The target pose of the effector

In [19]:
goal_pose = SE3(0.4, 0.5, 0.3)

In [22]:
# Добавление визуальных маркеров
goal_marker = rtb.models.URDF.Panda().links[7].collision[0]
goal_marker.T = goal_pose
env.add(goal_marker)

AttributeError: 'Cylinder' object has no attribute 'parent'

3. Identify obstacles

In [26]:
obstacles = [
    {'pos': [0.4, 0.2, 0.3], 'radius': 0.1},
    {'pos': [0.3, -0.1, 0.4], 'radius': 0.08}
]

4. Potential Functions for Stanford Arm

In [27]:
params = {
    'k_att': 0.8,      # Коэффициент притяжения
    'k_rep': 0.5,      # Коэффициент отталкивания
    'd_safe': 0.15,    # Радиус влияния препятствий
    'step_size': 0.05, # Шаг движения
    'max_steps': 500   # Максимальное число итераций
}


In [28]:

def compute_forces(q):
    current_pose = robot.fkine(q)
    pos = current_pose.t
    
    # Притягивающая сила
    f_att = params['k_att'] * (goal_pose.t - pos)
    
    # Отталкивающие силы
    f_rep = np.zeros(3)
    for obs in obstacles:
        delta = pos - obs['pos']
        dist = np.linalg.norm(delta)
        if dist < obs['radius'] + params['d_safe']:
            dir_vec = delta / (dist + 1e-6)
            rep_mag = params['k_rep'] * (1/(dist - obs['radius']) - 1/params['d_safe'])
            f_rep += rep_mag * dir_vec
    
    return f_att + f_rep

5. Numerical calculation of the gradient

In [8]:
# Начальная конфигурация (по обратной кинематике)
q_start = robot.ikine_LM(SE3(start_xyz))[0]
q = q_start.copy()
path = [robot.fkine(q).t[:2]]

for i in range(200):
    pos = robot.fkine(q).t
    pos2d = pos[:2]
    # Суммарная сила
    f_att = attractive_force(pos2d, goal_xyz[:2])
    f_rep = repulsive_force(pos2d)
    f_total = f_att + f_rep
    # Перевод силы в приращение по конфигурации через якобиан
    J = robot.jacobe(q)[:2, :]  # 2x6 якобиан по x,y
    dq = np.linalg.pinv(J) @ f_total * 0.1
    q += dq
    path.append(robot.fkine(q).t[:2])
    # Остановка при достижении цели
    if np.linalg.norm(pos2d - goal_xyz[:2]) < 0.01:
        break


TypeError: 'IKSolution' object is not subscriptable

6. Trajectory planning

In [ ]:
path = []
q = robot.ikine_LM(start_pose).q
robot.q = q

for _ in range(params['max_steps']):
    # Расчет сил и преобразование в скорость суставов
    force = compute_forces(q)
    J = robot.jacobe(q)[:3,:]
    dq = np.linalg.pinv(J) @ force * params['step_size']
    
    # Обновление конфигурации
    q += dq
    path.append(q.copy())
    robot.q = q
    env.step(0.05)
    
    # Проверка достижения цели
    if np.linalg.norm(robot.fkine(q).t - goal_pose.t) < 0.02:
        break

7. Visualization

In [ ]:
input("Нажмите Enter для воспроизведения траектории...")
env.reset()
for q_point in path:
    robot.q = q_point
    env.step(0.05)

3.7.5


In [ ]:
import roboticstoolbox as rtb
# Создаём backend для matplotlib

# Создаем backend PyPlot (важно: используем PyPlot.PyPlot, а не просто PyPlot())
pyplot = rtb.backends.PyPlot.PyPlot()
pyplot.add(robot)

# Анимация движения по рассчитанному пути
for q in path:
    robot.q = q
    pyplot.step(dt=0.05)

TypeError: argument must be scalar or ndarray

In [36]:
import matplotlib.pyplot as plt
import numpy as np

# Анимация движения робота по рассчитанному пути
robot.plot(path, backend='pyplot', dt=0.05)

# 3D-график траектории эффектора и препятствий
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Траектория эффектора
ee_path = np.array([robot.fkine(q).t for q in path])
ax.plot(ee_path[:, 0], ee_path[:, 1], ee_path[:, 2], 'b-', lw=2, label='Траектория эффектора')

# Визуализация препятствий (предполагается, что obstacles — список словарей с 'center' и 'radius')
for obs in obstacles:
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = obs['center'][0] + obs['radius'] * np.cos(u) * np.sin(v)
    y = obs['center'][1] + obs['radius'] * np.sin(u) * np.sin(v)
    z = obs['center'][2] + obs['radius'] * np.cos(v)
    ax.plot_surface(x, y, z, color='r', alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Траектория Stanford Arm с учётом препятствий')
ax.legend()
plt.show()

TypeError: argument must be scalar or ndarray

In [ ]:


# --- Траектория и препятствия ---

# --- График траектории эффектора и препятствий ---
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Траектория эффектора
ee_path = np.array([robot.fkine(q).t for q in path])
ax.plot(ee_path[:, 0], ee_path[:, 1], ee_path[:, 2], 'b-', lw=2, label='Траектория Эффектора')

# Препятствия
for obs in obstacles:
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = obs['center'][0] + obs['radius'] * np.cos(u) * np.sin(v)
    y = obs['center'][1] + obs['radius'] * np.sin(u) * np.sin(v)
    z = obs['center'][2] + obs['radius'] * np.cos(v)
    ax.plot_surface(x, y, z, color='r', alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Траектория Stanford Arm с учётом препятствий')
ax.legend()
plt.show()

TypeError: argument must be scalar or ndarray

In [ ]:

# Анимация движения
robot.plot(path, backend='swift', dt=0.05)

# График траектории в пространстве
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111, projection='3d')

# Траектория эффектора
ee_path = np.array([robot.fkine(q).t for q in path])
ax.plot(ee_path[:,0], ee_path[:,1], ee_path[:,2], 'b-', lw=2)

# Препятствия
for obs in obstacles:
    u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    x = obs['center'][0] + obs['radius']*np.cos(u)*np.sin(v)
    y = obs['center'][1] + obs['radius']*np.sin(u)*np.sin(v)
    z = obs['center'][2] + obs['radius']*np.cos(v)
    ax.plot_surface(x, y, z, color='r', alpha=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.title('Траектория Stanford Arm с учётом препятствий')
plt.show()

TypeError: 'module' object is not callable